# Real Time Premium Check

In [17]:
# Exchange Rate > EXR in late codes
# Memo: 환율이 낮을 때 원을 달러로 바꾸고 환율이 높을 때 달러를 원으로

import requests
import json

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

def upbit_get_usd_krw():
    url = 'https://quotation-api-cdn.dunamu.com/v1/forex/recent?codes=FRX.KRWUSD'
    exchange =requests.get(url, headers=headers).json()
    return exchange[0]['basePrice']

# print(upbit_get_usd_krw())


1188.0


In [32]:
 # Target Ticker : XLM

import ccxt
import pyupbit

def get_premium_XLM():
    binance = ccxt.binance()
    # ticker = binance.fetch_ticker('XLM/USDT')
    # print(ticker['open'], ticker['high'], ticker['low'], ticker['close'])

    EXR = upbit_get_usd_krw()

    orderbook_us = binance.fetch_order_book('XLM/USDT')
    # for ask in orderbook_us['asks']:
    #     print(ask[0]*EXR,ask[1])
    us_ask = orderbook_us['asks'][0][0]*EXR
    # print('us_ask: ',us_ask)

    orderbook_kr = pyupbit.get_orderbook('KRW-XLM')
    # for ask in orderbook_kr['orderbook_units']:
    #    print(ask)
    kr_bid = orderbook_kr['orderbook_units'][0]['bid_price']
    # print('kr_bid: ',kr_bid)

    prem = (kr_bid-us_ask)/kr_bid
    
    return (prem,kr_bid,us_ask)

ret = get_premium_XLM()
print(ret[0],ret[1],ret[2])
# print(prem)

# markets = binance.fetch_tickers()
# print(markets.keys())
# print('XLM/USDT' in markets.keys())

0.02259999999999991 330.0 322.54200000000003
